<a href="https://colab.research.google.com/github/guidogagl/physioex/blob/main/examples/colab_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# mount your personal gdrive folder inside google colab server
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# follow the guide on https://guidogagl.github.io/physioex/
# clone your forked repo
!git clone https://github.com/guidogagl/physioex.git
%cd physioex

!git pull origin main

fatal: destination path 'physioex' already exists and is not an empty directory.
/content/drive/MyDrive/Thesis/physioex


In [ ]:
# install the library in development mode
!pip install -e .

In [ ]:
# train the model proposed by chambon2018 on the DREEM dataset ( DODH version )
!train -e chambon2018 --dataset dreem --version dodh -vci 30 --sequence_lenght 3 -nj 2 --checkpoint "models/cel/chambon2018/seqlen=3/dreem/dodh/"